# Deploy model
**Important**: Change the kernel to *PROJECT_NAME local*. You can do this from the *Kernel* menu under *Change kernel*. You cannot deploy the model using the *PROJECT_NAME docker* kernel.

In [ ]:
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
import pandas as pd
import numpy as np
import imp
import pickle
import os
import sys
import json

In [ ]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.timeseries.deploy-model','true')

Enter the name of the model to deploy.

In [3]:
model_name = "linear_regression"

Load the test dataset and retain just one row. This record will be used to create and input schema for the web service. It will also allow us to simulate invoking the web service with features for one hour period and generating a demand forecast for this hour.

In [4]:
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']
test_df = pd.read_csv(os.path.join(aml_dir, 'nyc_demand_test.csv'), parse_dates=['timeStamp'])
test_df = test_df.drop(['demand', 'timeStamp'], axis=1).copy().iloc[[0]]
test_df

,precip,temp,hour,month,dayofweek,temp_lag1,temp_lag2,temp_lag3,temp_lag4,temp_lag5,temp_lag6,demand_lag1,demand_lag2,demand_lag3,demand_lag4,demand_lag5,demand_lag6
0,0.0,74.63,0,6,4,75.1,75.72,76.72,75.85,77.36,80.92,6912.7,7332.625,7576.558,7603.008,7788.292,8102.142


Load model from disk and transfer it to the working directory. 

In [5]:
with open(os.path.join(aml_dir, model_name + '.pkl'), 'rb') as f:
    mod = pickle.load(f)

with open('model_deploy.pkl', 'wb') as f:
    pickle.dump(mod, f)

Check model object has loaded as expected.

In [6]:
mod

Pipeline(steps=[('onehot', OneHotEncoder(categorical_features=[2, 3, 4], dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('rfecv', RFECV(cv=TimeSeriesSplit(n_splits=3),
   estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
   n_jobs=-1, scoring='neg_mean_squared_error', step=1, verbose=2))])

Apply model to predict test record

In [7]:
np.asscalar(mod.predict(test_df))

6501.258774578936

### Author a realtime web service

Create a score.py script which implements the scoring function to run inside the web service. Change model_name variable as required.

In [8]:
%%writefile score.py
# The init and run functions will load and score the input using the saved model.
# The score.py file will be included in the web service deployment package.
def init():
    import pickle
    import os
    global model
    
    with open('model_deploy.pkl', 'rb') as f:
        model = pickle.load(f)
    
def run(input_df):
    input_df = input_df[['precip', 'temp', 'hour', 'month', 'dayofweek',
        'temp_lag1', 'temp_lag2', 'temp_lag3', 'temp_lag4', 'temp_lag5',
        'temp_lag6', 'demand_lag1', 'demand_lag2', 'demand_lag3',
        'demand_lag4', 'demand_lag5', 'demand_lag6']]
    try:
        if (input_df.shape != (1,17)):
            return 'Bad imput: Expecting dataframe of shape (1,17)'
        else:
            pred = model.predict(input_df)
            return int(pred)
    except Exception as e:
        return(str(e))

Writing score.py


This script will be written to your current working directory:

In [ ]:
os.getcwd()

#### Test the *init* and *run* functions

In [ ]:
import score
imp.reload(score)

In [11]:
score.init()
score.run(test_df)

6501

#### Create web service schema
The web service schema provides details on the required structure of the input data as well as the data types of each column.

In [12]:
inputs = {"input_df": SampleDefinition(DataTypes.PANDAS, test_df)}
generate_schema(run_func=score.run, inputs=inputs, filepath='service_schema.json')

{'input': {'input_df': {'internal': 'gANjYXp1cmVtbC5hcGkuc2NoZW1hLnBhbmRhc1V0aWwKUGFuZGFzU2NoZW1hCnEAKYFxAX1xAihYBQAAAHNoYXBlcQNLAUsRhnEEWAwAAABjb2x1bW5fbmFtZXNxBV1xBihYBgAAAHByZWNpcHEHWAQAAAB0ZW1wcQhYBAAAAGhvdXJxCVgFAAAAbW9udGhxClgJAAAAZGF5b2Z3ZWVrcQtYCQAAAHRlbXBfbGFnMXEMWAkAAAB0ZW1wX2xhZzJxDVgJAAAAdGVtcF9sYWczcQ5YCQAAAHRlbXBfbGFnNHEPWAkAAAB0ZW1wX2xhZzVxEFgJAAAAdGVtcF9sYWc2cRFYCwAAAGRlbWFuZF9sYWcxcRJYCwAAAGRlbWFuZF9sYWcycRNYCwAAAGRlbWFuZF9sYWczcRRYCwAAAGRlbWFuZF9sYWc0cRVYCwAAAGRlbWFuZF9sYWc1cRZYCwAAAGRlbWFuZF9sYWc2cRdlWAwAAABjb2x1bW5fdHlwZXNxGF1xGShjbnVtcHkKZHR5cGUKcRpYAgAAAGY4cRtLAEsBh3EcUnEdKEsDWAEAAAA8cR5OTk5K/////0r/////SwB0cR9iaB1oGlgCAAAAaThxIEsASwGHcSFScSIoSwNoHk5OTkr/////Sv////9LAHRxI2JoImgiaB1oHWgdaB1oHWgdaB1oHWgdaB1oHWgdZVgKAAAAc2NoZW1hX21hcHEkfXElKGgRaB1oCmgiaAtoImgXaB1oCWgiaA9oHWgMaB1oFmgdaA5oHWgSaB1oDWgdaAdoHWgVaB1oE2gdaAhoHWgUaB1oEGgddXViLg==',
   'swagger': {'example': [{'dayofweek': 4,
      'demand_lag1': 6912.7,
      'demand_lag2': 7332.625,
      'demand_lag3': 757

#### Deploy the web service
The command below deploys a web service names "demandforecast", with input schema defined by "service_schema.json". The web service runs "score.py" which scores the input data using the model "model_deploy.pkl". This may take a few minutes.

In [ ]:
!az ml service create realtime -f score.py -m model_deploy.pkl -s service_schema.json -n demandforecast -r python

Check web service is running.

In [ ]:
!az ml service show realtime -i demandforecast

Test the web service is working by invoking it with a test record.

In [ ]:
!az ml service run realtime -i demandforecast -d "{\"input_df\": [{\"hour\": 0, \"month\": 6, \"demand_lag3\": 7576.558, \"temp_lag5\": 77.36, \"temp\": 74.63, \"demand_lag1\": 6912.7, \"demand_lag5\": 7788.292, \"temp_lag6\": 80.92, \"temp_lag3\": 76.72, \"demand_lag6\": 8102.142, \"temp_lag4\": 75.85, \"precip\": 0.0, \"temp_lag2\": 75.72, \"demand_lag2\": 7332.625, \"temp_lag1\": 75.1, \"demand_lag4\": 7603.008, \"dayofweek\": 4}]}"

#### Delete the web service

In [ ]:
!az ml service delete realtime --id=demandforecast